In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
from IPython.core.display import display, HTML


In [5]:
tiktok_df = pd.read_csv('../data/song_data.csv')

In [6]:
tiktok_df

,acousticness,artists,danceability,duration_ms,energy,explicit,genres,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.060100,"['City Morgue', 'ZillaKami', 'SosMula', 'Denze...",0.670,202613,0.731,NaN,['dark trap'],5hOnCHMwQdXOd4qfAWEhKP,0.000000,0,0.2160,-5.474,1,DRAINO (feat. Denzel Curry),57,12/13/19,0.0509,115.051,0.211,2019
1,0.004380,"['City Morgue', 'ZillaKami']",0.586,121277,0.817,NaN,['dark trap'],0NFCrBf6tnaDTZ5xOWjHCI,0.000000,4,0.2760,-7.257,1,YELLOW PISS,55,7/31/20,0.1340,87.496,0.728,2020
2,0.096600,"['City Morgue', 'ZillaKami', 'SosMula']",0.542,102587,0.908,NaN,['dark trap'],0T6wsLOq4GmZL4HtOseM4k,0.000000,1,0.1340,-2.857,1,SHINNERS13,54,10/12/18,0.0575,78.048,0.509,2018
3,0.093800,"['City Morgue', 'ZillaKami', 'SosMula']",0.828,117320,0.642,NaN,['dark trap'],6dIMYXE0iAoXfjQGU1aFdy,0.000000,2,0.1080,-5.160,0,SCREAMING AT THE RAIN,54,12/13/19,0.3000,89.972,0.104,2019
4,0.051100,"['City Morgue', 'ZillaKami']",0.764,83237,0.818,NaN,['dark trap'],4uUUPTF8566PHM1dZqH5Kb,0.026900,1,0.1590,-5.588,0,YOU CAN SMD,53,7/31/20,0.1900,175.136,0.370,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171042,0.000187,"['New World Sound', 'Thomas Newson']",0.622,287440,0.952,NaN,"['dutch house', 'electro house', 'melbourne bo...",1VAr8blNNsZLYd64tUih9f,0.330000,1,0.0431,-5.306,1,Flute - Original Mix,16,10/17/14,0.0477,128.008,0.111,2014
171043,0.124000,"['Prezioso', 'LIZOT', 'New World Sound', 'Shib...",0.735,161538,0.910,NaN,['italo dance'],2pqTSgyJPmJ5nOwgnftTr8,0.000063,8,0.2380,-4.559,0,Bounce Around,33,12/16/22,0.0767,130.074,0.557,2022
171044,0.039100,"['Sped Up Guys', 'Kiggo']",0.719,149272,0.832,NaN,['sped up'],5Vk9FO80ypBpWEeDe1kBLz,0.003890,3,0.1430,-4.622,0,Dancin (Sped Up),32,12/14/22,0.0404,157.186,0.595,2022
171045,0.046300,"['Nightcore Remix Guys', 'Sped Up Guys']",0.594,240230,0.824,NaN,['sped up'],6ReuTa17QGZwy0FjSHQm43,0.000000,3,0.0688,-5.223,1,I Gotta Feeling - Sped Up,37,2/15/23,0.0325,154.882,0.685,2023


In [7]:
spotify_df = pd.read_csv('../data/Merged_DataFrame.csv')

In [8]:
spotify_df

,acousticness,artists,danceability,duration_ms,energy,explicit,genres,id,instrumentalness,key,...,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,Unnamed: 0
0,0.06010,"['City Morgue', 'ZillaKami', 'SosMula', 'Denze...",0.670,202613,0.7310,NaN,['dark trap'],5hOnCHMwQdXOd4qfAWEhKP,0.000000,0,...,-5.474,1,DRAINO (feat. Denzel Curry),57,12/13/19,0.0509,115.051,0.2110,2019,NaN
1,0.00438,"['City Morgue', 'ZillaKami']",0.586,121277,0.8170,NaN,['dark trap'],0NFCrBf6tnaDTZ5xOWjHCI,0.000000,4,...,-7.257,1,YELLOW PISS,55,7/31/20,0.1340,87.496,0.7280,2020,NaN
2,0.09660,"['City Morgue', 'ZillaKami', 'SosMula']",0.542,102587,0.9080,NaN,['dark trap'],0T6wsLOq4GmZL4HtOseM4k,0.000000,1,...,-2.857,1,SHINNERS13,54,10/12/18,0.0575,78.048,0.5090,2018,NaN
3,0.09380,"['City Morgue', 'ZillaKami', 'SosMula']",0.828,117320,0.6420,NaN,['dark trap'],6dIMYXE0iAoXfjQGU1aFdy,0.000000,2,...,-5.160,0,SCREAMING AT THE RAIN,54,12/13/19,0.3000,89.972,0.1040,2019,NaN
4,0.05110,"['City Morgue', 'ZillaKami']",0.764,83237,0.8180,NaN,['dark trap'],4uUUPTF8566PHM1dZqH5Kb,0.026900,1,...,-5.588,0,YOU CAN SMD,53,7/31/20,0.1900,175.136,0.3700,2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339634,0.64300,['Kelly Clarkson'],0.481,205787,0.3680,0.0,NaN,5Y5b0wokJqZxaefDVRm43v,0.000000,7,...,-8.310,0,Born to Die,58,2020-03-13,0.0300,140.384,0.5070,2020,168587.0
339635,0.67000,['JoJo'],0.661,173760,0.5800,1.0,NaN,7wrBKecywD6TUuHQl3wjmj,0.000055,0,...,-7.718,1,Man,63,2020-03-13,0.1240,72.009,0.5660,2020,168588.0
339636,0.97900,['S.J Morgan'],0.502,160125,0.0355,0.0,NaN,67zZV7JzIx9Wcf65Ztt4CJ,0.867000,0,...,-26.940,1,Rivers,66,2020-02-28,0.0379,160.033,0.2820,2020,168589.0
339637,0.67200,['Childish Gambino'],0.174,179387,0.0466,0.0,NaN,4Ksj9mKfsYC5b8v8Ey3c8I,0.196000,3,...,-18.458,1,0.00,62,2020-03-22,0.0495,116.713,0.0337,2020,168590.0


In [9]:
spotify_df = spotify_df.dropna(axis = 1)

spotify_df = spotify_df.dropna()

In [10]:
tiktok_df

,acousticness,artists,danceability,duration_ms,energy,explicit,genres,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.060100,"['City Morgue', 'ZillaKami', 'SosMula', 'Denze...",0.670,202613,0.731,NaN,['dark trap'],5hOnCHMwQdXOd4qfAWEhKP,0.000000,0,0.2160,-5.474,1,DRAINO (feat. Denzel Curry),57,12/13/19,0.0509,115.051,0.211,2019
1,0.004380,"['City Morgue', 'ZillaKami']",0.586,121277,0.817,NaN,['dark trap'],0NFCrBf6tnaDTZ5xOWjHCI,0.000000,4,0.2760,-7.257,1,YELLOW PISS,55,7/31/20,0.1340,87.496,0.728,2020
2,0.096600,"['City Morgue', 'ZillaKami', 'SosMula']",0.542,102587,0.908,NaN,['dark trap'],0T6wsLOq4GmZL4HtOseM4k,0.000000,1,0.1340,-2.857,1,SHINNERS13,54,10/12/18,0.0575,78.048,0.509,2018
3,0.093800,"['City Morgue', 'ZillaKami', 'SosMula']",0.828,117320,0.642,NaN,['dark trap'],6dIMYXE0iAoXfjQGU1aFdy,0.000000,2,0.1080,-5.160,0,SCREAMING AT THE RAIN,54,12/13/19,0.3000,89.972,0.104,2019
4,0.051100,"['City Morgue', 'ZillaKami']",0.764,83237,0.818,NaN,['dark trap'],4uUUPTF8566PHM1dZqH5Kb,0.026900,1,0.1590,-5.588,0,YOU CAN SMD,53,7/31/20,0.1900,175.136,0.370,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171042,0.000187,"['New World Sound', 'Thomas Newson']",0.622,287440,0.952,NaN,"['dutch house', 'electro house', 'melbourne bo...",1VAr8blNNsZLYd64tUih9f,0.330000,1,0.0431,-5.306,1,Flute - Original Mix,16,10/17/14,0.0477,128.008,0.111,2014
171043,0.124000,"['Prezioso', 'LIZOT', 'New World Sound', 'Shib...",0.735,161538,0.910,NaN,['italo dance'],2pqTSgyJPmJ5nOwgnftTr8,0.000063,8,0.2380,-4.559,0,Bounce Around,33,12/16/22,0.0767,130.074,0.557,2022
171044,0.039100,"['Sped Up Guys', 'Kiggo']",0.719,149272,0.832,NaN,['sped up'],5Vk9FO80ypBpWEeDe1kBLz,0.003890,3,0.1430,-4.622,0,Dancin (Sped Up),32,12/14/22,0.0404,157.186,0.595,2022
171045,0.046300,"['Nightcore Remix Guys', 'Sped Up Guys']",0.594,240230,0.824,NaN,['sped up'],6ReuTa17QGZwy0FjSHQm43,0.000000,3,0.0688,-5.223,1,I Gotta Feeling - Sped Up,37,2/15/23,0.0325,154.882,0.685,2023


In [11]:
tiktok_df = tiktok_df.drop('explicit', axis = 1)

tiktok_df = tiktok_df.dropna()

In [12]:
tiktok_df.head()

,acousticness,artists,danceability,duration_ms,energy,genres,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.06010,"['City Morgue', 'ZillaKami', 'SosMula', 'Denze...",0.670,202613,0.731,['dark trap'],5hOnCHMwQdXOd4qfAWEhKP,0.0000,0,0.216,-5.474,1,DRAINO (feat. Denzel Curry),57,12/13/19,0.0509,115.051,0.211,2019
1,0.00438,"['City Morgue', 'ZillaKami']",0.586,121277,0.817,['dark trap'],0NFCrBf6tnaDTZ5xOWjHCI,0.0000,4,0.276,-7.257,1,YELLOW PISS,55,7/31/20,0.1340,87.496,0.728,2020
2,0.09660,"['City Morgue', 'ZillaKami', 'SosMula']",0.542,102587,0.908,['dark trap'],0T6wsLOq4GmZL4HtOseM4k,0.0000,1,0.134,-2.857,1,SHINNERS13,54,10/12/18,0.0575,78.048,0.509,2018
3,0.09380,"['City Morgue', 'ZillaKami', 'SosMula']",0.828,117320,0.642,['dark trap'],6dIMYXE0iAoXfjQGU1aFdy,0.0000,2,0.108,-5.160,0,SCREAMING AT THE RAIN,54,12/13/19,0.3000,89.972,0.104,2019
4,0.05110,"['City Morgue', 'ZillaKami']",0.764,83237,0.818,['dark trap'],4uUUPTF8566PHM1dZqH5Kb,0.0269,1,0.159,-5.588,0,YOU CAN SMD,53,7/31/20,0.1900,175.136,0.370,2020


In [13]:
tiktok_df.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
genres               object
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

In [14]:
tiktok_df['genres_upd'] = tiktok_df['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [15]:
tiktok_df['genres_upd'].values[0][0]

'dark_trap'

In [16]:
tiktok_df['artists_upd_v1'] = tiktok_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [17]:
tiktok_df[tiktok_df['artists_upd_v1'].apply(lambda x: not x)].head(5)

,acousticness,artists,danceability,duration_ms,energy,genres,id,instrumentalness,key,liveness,...,mode,name,popularity,release_date,speechiness,tempo,valence,year,genres_upd,artists_upd_v1
153,0.128,"[""D'Evil""]",0.772,187580,0.723,"['desi hip hop', 'hindi hip hop', 'indian unde...",2G4ahExauuyWJUCsVpXUjj,0.000000,10,0.239,...,0,"Todun Taak - From ""Toofaan""",23,7/2/21,0.1900,96.025,0.868,2021,"[desi_hip_hop, hindi_hip_hop, indian_undergrou...",[]
1069,0.844,"[""Howlin' Wolf""]",0.661,188400,0.342,"['acoustic blues', 'blues', 'chicago blues', '...",2HUZVffVPXvqnrml0gXggp,0.000128,9,0.260,...,0,Smokestack Lightnin',60,3/1/58,0.0430,146.118,0.588,1958,"[acoustic_blues, blues, chicago_blues, delta_b...",[]
1070,0.669,"[""Howlin' Wolf""]",0.821,170000,0.376,"['acoustic blues', 'blues', 'chicago blues', '...",7j02rdE5RVtNcNMuLGY5SS,0.000014,0,0.106,...,1,Spoonful,56,1/11/62,0.0487,119.039,0.824,1962,"[acoustic_blues, blues, chicago_blues, delta_b...",[]
1071,0.379,"[""Howlin' Wolf""]",0.721,172827,0.620,"['acoustic blues', 'blues', 'chicago blues', '...",1a8vVeidOsauNu6rQmHBRS,0.000043,9,0.344,...,1,Killing Floor,47,1/11/66,0.0326,118.696,0.644,1966,"[acoustic_blues, blues, chicago_blues, delta_b...",[]
1072,0.761,"[""Howlin' Wolf""]",0.546,149400,0.185,"['acoustic blues', 'blues', 'chicago blues', '...",22o1qy3iZ7CyXUdK6BIf2n,0.000001,9,0.218,...,1,Little Red Rooster,44,1/11/62,0.0409,69.028,0.682,1962,"[acoustic_blues, blues, chicago_blues, delta_b...",[]


In [18]:
tiktok_df['artists_upd_v2'] = tiktok_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
tiktok_df['artists_upd'] = np.where(tiktok_df['artists_upd_v1'].apply(lambda x: not x), tiktok_df['artists_upd_v2'], tiktok_df['artists_upd_v1'] )

In [19]:
tiktok_df

,acousticness,artists,danceability,duration_ms,energy,genres,id,instrumentalness,key,liveness,...,popularity,release_date,speechiness,tempo,valence,year,genres_upd,artists_upd_v1,artists_upd_v2,artists_upd
0,0.060100,"['City Morgue', 'ZillaKami', 'SosMula', 'Denze...",0.670,202613,0.731,['dark trap'],5hOnCHMwQdXOd4qfAWEhKP,0.000000,0,0.2160,...,57,12/13/19,0.0509,115.051,0.211,2019,[dark_trap],"[City Morgue, ZillaKami, SosMula, Denzel Curry]",[],"[City Morgue, ZillaKami, SosMula, Denzel Curry]"
1,0.004380,"['City Morgue', 'ZillaKami']",0.586,121277,0.817,['dark trap'],0NFCrBf6tnaDTZ5xOWjHCI,0.000000,4,0.2760,...,55,7/31/20,0.1340,87.496,0.728,2020,[dark_trap],"[City Morgue, ZillaKami]",[],"[City Morgue, ZillaKami]"
2,0.096600,"['City Morgue', 'ZillaKami', 'SosMula']",0.542,102587,0.908,['dark trap'],0T6wsLOq4GmZL4HtOseM4k,0.000000,1,0.1340,...,54,10/12/18,0.0575,78.048,0.509,2018,[dark_trap],"[City Morgue, ZillaKami, SosMula]",[],"[City Morgue, ZillaKami, SosMula]"
3,0.093800,"['City Morgue', 'ZillaKami', 'SosMula']",0.828,117320,0.642,['dark trap'],6dIMYXE0iAoXfjQGU1aFdy,0.000000,2,0.1080,...,54,12/13/19,0.3000,89.972,0.104,2019,[dark_trap],"[City Morgue, ZillaKami, SosMula]",[],"[City Morgue, ZillaKami, SosMula]"
4,0.051100,"['City Morgue', 'ZillaKami']",0.764,83237,0.818,['dark trap'],4uUUPTF8566PHM1dZqH5Kb,0.026900,1,0.1590,...,53,7/31/20,0.1900,175.136,0.370,2020,[dark_trap],"[City Morgue, ZillaKami]",[],"[City Morgue, ZillaKami]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171042,0.000187,"['New World Sound', 'Thomas Newson']",0.622,287440,0.952,"['dutch house', 'electro house', 'melbourne bo...",1VAr8blNNsZLYd64tUih9f,0.330000,1,0.0431,...,16,10/17/14,0.0477,128.008,0.111,2014,"[dutch_house, electro_house, melbourne_bounce,...","[New World Sound, Thomas Newson]",[],"[New World Sound, Thomas Newson]"
171043,0.124000,"['Prezioso', 'LIZOT', 'New World Sound', 'Shib...",0.735,161538,0.910,['italo dance'],2pqTSgyJPmJ5nOwgnftTr8,0.000063,8,0.2380,...,33,12/16/22,0.0767,130.074,0.557,2022,[italo_dance],"[Prezioso, LIZOT, New World Sound, Shibui, Pao...",[],"[Prezioso, LIZOT, New World Sound, Shibui, Pao..."
171044,0.039100,"['Sped Up Guys', 'Kiggo']",0.719,149272,0.832,['sped up'],5Vk9FO80ypBpWEeDe1kBLz,0.003890,3,0.1430,...,32,12/14/22,0.0404,157.186,0.595,2022,[sped_up],"[Sped Up Guys, Kiggo]",[],"[Sped Up Guys, Kiggo]"
171045,0.046300,"['Nightcore Remix Guys', 'Sped Up Guys']",0.594,240230,0.824,['sped up'],6ReuTa17QGZwy0FjSHQm43,0.000000,3,0.0688,...,37,2/15/23,0.0325,154.882,0.685,2023,[sped_up],"[Nightcore Remix Guys, Sped Up Guys]",[],"[Nightcore Remix Guys, Sped Up Guys]"


In [20]:
#tiktok_df['artists_song'] = tiktok_df.apply(lambda row: row['artists_upd'][0]+row['name'],axis = 1)
tiktok_df['artists_song'] = tiktok_df.apply(lambda row: str(row['artists_upd'][0]) + str(row['name']), axis=1)

In [21]:
tiktok_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [22]:
tiktok_df.drop_duplicates('artists_song',inplace = True)

In [23]:
tiktok_df

,acousticness,artists,danceability,duration_ms,energy,genres,id,instrumentalness,key,liveness,...,release_date,speechiness,tempo,valence,year,genres_upd,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
116313,0.8070,['･ ･－･ ･－ ･･･ ･ －･･'],0.390,438130,0.2540,['broken transmission'],26trS5IUeIbXrKAuFMdzWT,0.38100,4,0.1210,...,7/22/20,0.0311,144.642,0.117,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･･－･･ ･ － ･･ － －－･ －－－
116317,0.7350,['･ ･－･ ･－ ･･･ ･ －･･'],0.345,524020,0.4420,['broken transmission'],4akqPnWyCOWL0QNxQsXsNk,0.88700,8,0.1070,...,2/5/20,0.0720,115.013,0.201,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･－－ ･ －･･ ･ ･－･･ ･－
116314,0.8080,['･ ･－･ ･－ ･･･ ･ －･･'],0.392,434721,0.2010,['broken transmission'],5xPFWA9a9cwnlywB7yPvxp,0.38400,4,0.1120,...,8/6/21,0.0312,144.909,0.126,2021,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･・-・・ ・ - ・・ - --・ ---
116315,0.8720,['･ ･－･ ･－ ･･･ ･ －･･'],0.329,356698,0.0797,['broken transmission'],1F9f5t7GZk7aJZNGZIbfqP,0.00889,7,0.1200,...,8/19/20,0.0459,131.692,0.154,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･-・・ ・- -・ -・-・ ・ -- ・ - --- ...
116316,0.9300,['･ ･－･ ･－ ･･･ ･ －･･'],0.182,320000,0.1530,['broken transmission'],0nL71ZAhZdoMvtGPNs8oHH,0.94400,11,0.0809,...,8/6/21,0.0377,95.760,0.232,2021,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･-・ --- ・・・ - ・ ・--・ --- -・ ・...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90318,0.5400,"[""Ben Jammin' Beats"", 'Illuzionary', 'Chill Se...",0.586,112208,0.4000,[],1doqnkw76v02noZLwzTfd0,0.89700,10,0.1040,...,12/8/23,0.0453,76.991,0.496,2023,[],"[ Beats"", , , ]",[Ben Jammin' Beats],"[ Beats"", , , ]","Beats"", Moods"
90276,0.2600,"[""Ben Jammin' Beats"", 'Illuzionary', 'Chill Se...",0.786,83143,0.3620,[],1xtHtr7oaTEWFElc83ffVH,0.58700,3,0.1440,...,11/24/23,0.0568,140.147,0.801,2023,[],"[ Beats"", , , ]",[Ben Jammin' Beats],"[ Beats"", , , ]","Beats"", After Hours"
46571,0.0428,"[""Tre' Amani"", 'Rodney Hollywood']",0.846,148118,0.5920,[],6vQK5arrwdS2rrpNJWJ3EI,0.00000,8,0.2560,...,6/3/22,0.0521,109.980,0.718,2022,[],"[ Amani"", ]",[Tre' Amani],"[ Amani"", ]","Amani"", lo$t kid."
46569,0.0908,"[""Tre' Amani"", 'Brent Faiyaz']",0.779,196204,0.4580,[],4Dg81Gxw4a1sBI6J6rtCEL,0.00000,3,0.0852,...,6/26/20,0.5790,137.299,0.559,2020,[],"[ Amani"", ]",[Tre' Amani],"[ Amani"", ]","Amani"", On Us (feat. Brent Faiyaz)"


In [24]:
artists_genres_consolidated = tiktok_df.groupby('id')['genres_upd'].apply(list).reset_index()
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [25]:
artists_genres_consolidated.head()

,id,genres_upd,consolidates_genre_lists
0,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[country_quebecois, rock_quebecois]"
1,000uPbEAb9tlFkCJ1DIOcw,"[[classic_swedish_pop, europop, swedish_pop]]","[europop, classic_swedish_pop, swedish_pop]"
2,000vyG8dpBoj3SHBbzPsxa,"[[doo-wop, rock-and-roll]]","[rock-and-roll, doo-wop]"
3,001BlLVP9bNnfYGDEeIznJ,"[[polish_hip_hop, polish_trap]]","[polish_trap, polish_hip_hop]"
4,001VVcybqoFwkm1jbvxL4j,"[[otacore, pixel]]","[pixel, otacore]"


In [26]:
tiktok_df = tiktok_df.merge(artists_genres_consolidated[['id','consolidates_genre_lists']], on = 'id',how = 'left')

In [27]:
tiktok_df.head()

,acousticness,artists,danceability,duration_ms,energy,genres,id,instrumentalness,key,liveness,...,speechiness,tempo,valence,year,genres_upd,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0.807,['･ ･－･ ･－ ･･･ ･ －･･'],0.390,438130,0.2540,['broken transmission'],26trS5IUeIbXrKAuFMdzWT,0.38100,4,0.1210,...,0.0311,144.642,0.117,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･･－･･ ･ － ･･ － －－･ －－－,[broken_transmission]
1,0.735,['･ ･－･ ･－ ･･･ ･ －･･'],0.345,524020,0.4420,['broken transmission'],4akqPnWyCOWL0QNxQsXsNk,0.88700,8,0.1070,...,0.0720,115.013,0.201,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･－－ ･ －･･ ･ ･－･･ ･－,[broken_transmission]
2,0.808,['･ ･－･ ･－ ･･･ ･ －･･'],0.392,434721,0.2010,['broken transmission'],5xPFWA9a9cwnlywB7yPvxp,0.38400,4,0.1120,...,0.0312,144.909,0.126,2021,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･・-・・ ・ - ・・ - --・ ---,[broken_transmission]
3,0.872,['･ ･－･ ･－ ･･･ ･ －･･'],0.329,356698,0.0797,['broken transmission'],1F9f5t7GZk7aJZNGZIbfqP,0.00889,7,0.1200,...,0.0459,131.692,0.154,2020,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･-・・ ・- -・ -・-・ ・ -- ・ - --- ...,[broken_transmission]
4,0.930,['･ ･－･ ･－ ･･･ ･ －･･'],0.182,320000,0.1530,['broken transmission'],0nL71ZAhZdoMvtGPNs8oHH,0.94400,11,0.0809,...,0.0377,95.760,0.232,2021,[broken_transmission],[･ ･－･ ･－ ･･･ ･ －･･],[],[･ ･－･ ･－ ･･･ ･ －･･],･ ･－･ ･－ ･･･ ･ －･･-・ --- ・・・ - ・ ・--・ --- -・ ・...,[broken_transmission]


In [28]:
tiktok_df['year'] = tiktok_df['release_date'].apply(lambda x: x.split('-')[0])
float_cols = tiktok_df.dtypes[tiktok_df.dtypes == 'float64'].index.values
ohe_cols = 'popularity'
tiktok_df['popularity'].describe()

count    166806.000000
mean         37.847805
std          11.201919
min           0.000000
25%          32.000000
50%          39.000000
75%          45.000000
max          80.000000
Name: popularity, dtype: float64

In [29]:
tiktok_df['popularity_red'] = tiktok_df['popularity'].apply(lambda x: int(x/5))

In [30]:
tiktok_df['consolidates_genre_lists'] = tiktok_df['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [31]:
def ohe_prep(df, column, new_name): 
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [32]:
def create_feature_set(df, float_cols):

    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray(), columns=['genre' + "|" + i for i in tfidf.get_feature_names()])
    genre_df.reset_index(drop=True, inplace=True)


    year_ohe = ohe_prep(df, 'year', 'year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red', 'pop') * 0.15


    floats = df[float_cols].reset_index(drop=True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) * 0.2


    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis=1)
    final['id'] = df['id'].values

    return final

In [33]:
complete_feature_set = create_feature_set(tiktok_df, float_cols=float_cols)

In [34]:
complete_feature_set

,genre|21st_century_classical,genre|432hz,genre|48g,genre|528hz,genre|5th_gen_k,genre|5th_wave_emo,genre|8d,genre|_brasileira,genre|_hip_hop,genre|_indie,...,year|9/9/72,year|9/9/76,year|9/9/89,year|9/9/91,year|9/9/93,year|9/9/96,year|9/9/97,year|9/9/98,year|9/9/99,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26trS5IUeIbXrKAuFMdzWT
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4akqPnWyCOWL0QNxQsXsNk
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5xPFWA9a9cwnlywB7yPvxp
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1F9f5t7GZk7aJZNGZIbfqP
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0nL71ZAhZdoMvtGPNs8oHH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1doqnkw76v02noZLwzTfd0
166802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1xtHtr7oaTEWFElc83ffVH
166803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6vQK5arrwdS2rrpNJWJ3EI
166804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4Dg81Gxw4a1sBI6J6rtCEL


In [35]:
#client id and secret for my application
client_id = 'id'
client_secret= 'secret'

In [36]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [37]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [38]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='https://google.com/callback') #maybe 8888 localhost

In [39]:
sp = spotipy.Spotify(auth=token)

In [40]:

id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
#    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [41]:
id_name

{'Night': '35T02TTUDYoNfl8c7WysQ6',
 'Music from Indieland ': '6a7eYcgNsXyg6V7JOhgZ4X',
 'Big Bangers (2)': '1rSBs6RBFXF4i4dOHwJlyr',
 'Feels': '5z7kttlVMfhgm6mWtU7U6b',
 'drive and power': '0ZoxWypHG3rb7gf6zhvgDW',
 'Art': '3RPj8fBVdgzwnTkRkRllcy',
 'RAWR.EXE': '45vrD2zSh237WkdRtTZf9D',
 'Hits different': '7bjBbM4uMdyxbunbA2i903'}

In [42]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [43]:
id_name

{'Night': '35T02TTUDYoNfl8c7WysQ6',
 'Music from Indieland ': '6a7eYcgNsXyg6V7JOhgZ4X',
 'Big Bangers (2)': '1rSBs6RBFXF4i4dOHwJlyr',
 'Feels': '5z7kttlVMfhgm6mWtU7U6b',
 'drive and power': '0ZoxWypHG3rb7gf6zhvgDW',
 'Art': '3RPj8fBVdgzwnTkRkRllcy',
 'RAWR.EXE': '45vrD2zSh237WkdRtTZf9D',
 'Hits different': '7bjBbM4uMdyxbunbA2i903'}

In [44]:
playlist = create_necessary_outputs('Music from Indieland ',id_name, spotify_df)

In [47]:
playlist

,artist,name,id,url,date_added
0,The Strokes,The End Has No End,0dy6iXYIF0piirySAzCBwF,https://i.scdn.co/image/ab67616d00001e020f3572...,2024-07-06 02:49:58+00:00
80,Wallows,Remember When,5naar7XewEOAjOywIp6Jjq,https://i.scdn.co/image/ab67616d00001e0284feca...,2024-07-06 02:49:58+00:00
60,Peach Pit,Alrighty Aphrodite,6OiRh4kttAs1YWglvTcYkB,https://i.scdn.co/image/ab67616d00001e0200b39b...,2024-07-06 02:49:58+00:00
63,Declan McKenna,Brazil,4sNG6zQBmtq7M8aeeKJRMQ,https://i.scdn.co/image/ab67616d00001e02ec8be2...,2024-07-06 02:49:58+00:00
67,Arctic Monkeys,Why'd You Only Call Me When You're High?,086myS9r57YsLbJpU0TgK9,https://i.scdn.co/image/ab67616d00001e024ae1c4...,2024-07-06 02:49:58+00:00
76,Last Dinosaurs,Sense,1UCeirBzcsC0Mk8Fnvl7Ix,https://i.scdn.co/image/ab67616d00001e02b38a6a...,2024-07-06 02:49:58+00:00
77,Last Dinosaurs,Italo Disco,1B2zrooBJ6Rc8NIUuS3VDh,https://i.scdn.co/image/ab67616d00001e02b38a6a...,2024-07-06 02:49:58+00:00
78,Kid Bloom,Electric U,1h3ODYcDJly0H53W2Ai42Q,https://i.scdn.co/image/ab67616d00001e02159b01...,2024-07-06 02:49:58+00:00
85,Declan McKenna,Make Me Your Queen,5QJuMhGWTUB4t9BqR7ndL3,https://i.scdn.co/image/ab67616d00001e02ec8be2...,2024-07-06 02:49:58+00:00
56,Turin Brakes,Underdog (Save Me),3WfnpB1e49F0gN18aaPw8Y,https://i.scdn.co/image/ab67616d00001e02d4ee53...,2024-07-06 02:49:58+00:00


In [49]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [50]:
complete_feature_set

,genre|21st_century_classical,genre|432hz,genre|48g,genre|528hz,genre|5th_gen_k,genre|5th_wave_emo,genre|8d,genre|_brasileira,genre|_hip_hop,genre|_indie,...,year|9/9/72,year|9/9/76,year|9/9/89,year|9/9/91,year|9/9/93,year|9/9/96,year|9/9/97,year|9/9/98,year|9/9/99,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26trS5IUeIbXrKAuFMdzWT
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4akqPnWyCOWL0QNxQsXsNk
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5xPFWA9a9cwnlywB7yPvxp
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1F9f5t7GZk7aJZNGZIbfqP
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0nL71ZAhZdoMvtGPNs8oHH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1doqnkw76v02noZLwzTfd0
166802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1xtHtr7oaTEWFElc83ffVH
166803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6vQK5arrwdS2rrpNJWJ3EI
166804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4Dg81Gxw4a1sBI6J6rtCEL


In [51]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist, 1.09)


In [52]:
complete_feature_set_playlist_vector.shape

(15731,)

In [53]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(200)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [54]:
top40 = generate_playlist_recos(tiktok_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

In [55]:
top40['priority'] = top40['popularity_red'] * top40['sim']
top40 = top40.drop(['acousticness', 'danceability', 'duration_ms', 'energy',
               'instrumentalness', 'key', 'liveness', 'artists_upd_v2',
               'url', 'genres', 'artists', 'loudness', 'popularity', 
               'speechiness', 'tempo', 'valence', 'year', 'id', 
                'artists_upd_v1', 'genres_upd', 'mode'], axis = 1)

In [56]:
top40 = top40.drop(['artists_song'], axis = 1)

In [57]:
top40 = top40.loc[:, ['name', 'artists_upd', 'consolidates_genre_lists', 'priority', 'release_date', 'popularity_red', 'sim']]

In [58]:
top40.sort_values(by='priority', ascending=False).head(40)

,name,artists_upd,consolidates_genre_lists,priority,release_date,popularity_red,sim
25701,Charlie's Inferno,[That Handsome Devil],[pov:_indie],6.843506,8/9/11,12,0.570292
75215,Blue Haired Boy,[Madilyn Mei],[pov:_indie],5.950823,7/29/22,11,0.540984
63551,Person in the Mirror,[Naethan Apollo],[pov:_indie],5.913954,4/22/22,11,0.537632
63549,You're Not Welcome,[Naethan Apollo],[pov:_indie],5.909630,3/17/23,11,0.537239
75211,The Milk Carton,[Madilyn Mei],[pov:_indie],5.824438,1/20/23,11,0.529494
92290,Passing Through (Can't the Future Just Wait),[Kaden MacKay],"[pixel, pov:_indie]",5.817800,12/11/21,13,0.447523
99373,Yandere,[Jazmin Bean],[pov:_indie],5.752889,10/30/20,11,0.522990
11703,I / Me / Myself,[Will Wood],"[pov:_indie, weirdcore]",5.703844,7/10/20,12,0.475320
75213,Partner in Crime,[Madilyn Mei],[pov:_indie],5.685180,10/27/23,11,0.516835
99376,Hello Kitty,[Jazmin Bean],[pov:_indie],5.662720,11/11/20,11,0.514793
